# Merging data in Python

This notebook shows how you can use the `pandas` library in Python to merge datasets. 

In order to merge those datasets they *must* have **data in common** - something to merge *on*. 

A common example might be merging two datasets with different information about geographical areas (for example population vs crimes). In this situation the data in common would be the names of the areas (or better still, the administrative codes for those areas).

We can do this with the `pandas` library.

## Import the `pandas` library

We import the `pandas` library first, and rename it `pd` to make it quicker to name later.

We also install `odfpy` which will allow us to handle an ODS spreadsheet format. 

In [ ]:
#import the pandas library which has functions to merge data
import pandas as pd
#install the odfpy library for ods files
!pip install odfpy

## Import the data

Now we need some data to merge. A common scenario in data journalism is needing to put events into context by bringing in extra data - for example population size.

We have that scenario for our story on FOI responses: although we know the number of FOIs refused by each body, and the reasons for those refusals, we don't actually know how many responses overall they received, and therefore which body had the highest or lowest *proportion* of requests refused.

Let's import the data from the [Freedom of Information statistics: April to June 2021 bulletin](https://www.gov.uk/government/statistics/freedom-of-information-statistics-april-to-june-2021/freedom-of-information-statistics-april-to-june-2021-bulletin) - specifically the sheet on refusals.

In [2]:
#store the URL of the file
odsurl = "https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1017270/foi-statistics-q2-2021-statistical-tables.ods"
#read the file at that URL, fetching the 12th sheet, starting at the 10th row and capturing 20 rows only
foidata = pd.read_excel(odsurl, sheet_name=11, header=9, nrows=20)
#check the first few rows
foidata.head()

,Government body,Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",S.24 - National security,S.26 - Defence,S.27 - International relations,S.28 - Relations within the United Kingdom,S.29 - The economy,...,S.36 - Prejudice to effective conduct of public affairs,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,Unnamed: 25
0,Attorney General's Office,9,0,0,0,0,0,1,0,0,...,3,0,0,2,1,1,0,0,0,NaN
1,Cabinet Office [note 4],180,41,1,9,14,1,11,1,0,...,14,7,1,46,9,2,26,0,0,NaN
2,"Department for Business, Energy and Industrial...",108,12,0,0,4,0,4,0,1,...,3,0,1,24,3,0,28,2,37,NaN
3,"Department for Digital, Culture, Media and Sport",39,5,0,1,0,0,0,0,0,...,7,1,0,17,1,0,5,0,0,NaN
4,Department for Education [note 4],124,13,0,0,0,0,0,0,0,...,15,0,0,81,7,3,5,3,4,NaN


## Import the second dataset

Now let's import the data on requests overall. This is on the third sheet. 

In [3]:
#read the file at that URL, fetching the 3rd sheet, starting at the 10th row and capturing 20 rows only
requestsdata = pd.read_excel(odsurl, sheet_name=2, header=9, nrows=20)
#check the first few rows
requestsdata

,Government body,Total requests received,Total requests processed,"Total requests ""On hold"" or lapsed [note 2]",Total requests still being processed,Total requests handled under EIRs [note 3]
0,Attorney General's Office,50,48,0,2,0
1,Cabinet Office [note 4],705,635,0,70,0
2,"Department for Business, Energy and Industrial...",366,330,0,36,68
3,"Department for Digital, Culture, Media and Sport",198,158,0,40,1
4,Department for Education [note 4],596,548,0,48,3
5,"Department for Environment, Food and Rural Aff...",239,236,0,3,94
6,Department for International Trade,126,122,0,4,3
7,Department for Transport [note 4],759,728,0,31,27
8,Department for Work and Pensions,613,605,0,8,1
9,Department of Health and Social Care,715,599,0,116,1


## Merge on a common field

In order to merge the two datasets they must both have a field (column) in common. In this case, both have the name of the government body. 

We can use [the `merge()` function](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) in `pandas` to do this. This function has various ingredients. These are the ones we are going to use:

* `left=` specifies the dataframe which will be to the left of the resulting merged dataframe
* `right=` specifies the dataframe which will end up to the right of the resulting merged dataframe
* `left_on=` specifies the column name from the first (left) dataframe which you are matching on
* `right_on=` specifies the column name from the second (right) dataframe which you are matching on
* `how=` specifies the type of join you want to make: `inner` specifies that you only want to keep rows where both dataframes contain the same value in the matching column, whereas `left` specifies you want to keep all the data from the left dataframe, even if there's no match in the right dataframe, and you can also specify `outer` or `right` to get the opposite results to the others.

As it happens we don't *have* to use all of these options because the column names happen to be the same in both dataframes, and the 20 bodies all match up, but it's good to identify them all as most of the time you will need to. 

In [5]:
#merge the two dataframes on the 'Government body' column 
pd.merge(left=requestsdata, 
        right=foidata, 
        left_on="Government body", 
        right_on="Government body", 
        how="inner")

,Government body,Total requests received,Total requests processed,"Total requests ""On hold"" or lapsed [note 2]",Total requests still being processed,Total requests handled under EIRs [note 3],Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",...,S.36 - Prejudice to effective conduct of public affairs,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,Unnamed: 25
0,Attorney General's Office,50,48,0,2,0,9,0,0,0,...,3,0,0,2,1,1,0,0,0,NaN
1,Cabinet Office [note 4],705,635,0,70,0,180,41,1,9,...,14,7,1,46,9,2,26,0,0,NaN
2,"Department for Business, Energy and Industrial...",366,330,0,36,68,108,12,0,0,...,3,0,1,24,3,0,28,2,37,NaN
3,"Department for Digital, Culture, Media and Sport",198,158,0,40,1,39,5,0,1,...,7,1,0,17,1,0,5,0,0,NaN
4,Department for Education [note 4],596,548,0,48,3,124,13,0,0,...,15,0,0,81,7,3,5,3,4,NaN
5,"Department for Environment, Food and Rural Aff...",239,236,0,3,94,98,2,0,1,...,1,0,0,23,3,0,5,0,60,NaN
6,Department for International Trade,126,122,0,4,3,46,6,0,0,...,3,1,0,25,7,0,11,0,0,NaN
7,Department for Transport [note 4],759,728,0,31,27,163,17,0,2,...,8,0,2,79,9,1,23,5,7,NaN
8,Department for Work and Pensions,613,605,0,8,1,99,19,0,0,...,0,1,0,53,5,0,8,4,1,NaN
9,Department of Health and Social Care,715,599,0,116,1,128,25,0,0,...,1,0,0,34,2,1,33,0,1,NaN


### Store in a variable

That works nicely - let's store it in a variable and whittle it down.

In [6]:
#merge the two dataframes on the 'Government body' column 
mergeddf = pd.merge(left=requestsdata, 
                    right=foidata, 
                    left_on="Government body", 
                    right_on="Government body", 
                    how="inner")
mergeddf.columns

Index(['Government body', 'Total requests received',
       'Total requests processed',
       ' Total requests "On hold" or lapsed [note 2]',
       'Total requests still being processed',
       'Total requests handled under EIRs [note 3]',
       'Total requests where one or more exemptions / exceptions were applied [note 18] [note 19]',
       'S.22 - Information intended for future publication',
       'S. 22A - Research intended for future publication',
       'S.23 - Information supplied by, or relating to, bodies dealing with security matters',
       'S.24 - National security', 'S.26 - Defence',
       'S.27 - International relations',
       'S.28 - Relations within the United Kingdom', 'S.29 - The economy',
       'S.30 - Investigations and proceedings conducted by public authorities',
       'S.31 - Law enforcement', 'S.32 - Court records, etc.',
       'S.33 - Audit functions', 'S.34 - Parliamentary privilege',
       'S.35 - Formulation of Government policy, etc.',
      

In [12]:
mergeddf.columns = ['Government body', 'Total requests received',
       'Total requests processed',
       ' Total requests "On hold" or lapsed [note 2]',
       'Total requests still being processed',
       'Total requests handled under EIRs [note 3]',
       'Total requests where one or more exemptions / exceptions were applied [note 18] [note 19]',
       'S.22 - Information intended for future publication',
       'S. 22A - Research intended for future publication',
       'S.23 - Information supplied by, or relating to, bodies dealing with security matters',
       'S.24 - National security', 'S.26 - Defence',
       'S.27 - International relations',
       'S.28 - Relations within the United Kingdom', 'S.29 - The economy',
       'S.30 - Investigations and proceedings conducted by public authorities',
       'S.31 - Law enforcement', 'S.32 - Court records, etc.',
       'S.33 - Audit functions', 'S.34 - Parliamentary privilege',
       'S.35 - Formulation of Government policy, etc.',
       'S.36 - Prejudice to effective conduct of public affairs',
       'S.37 - Communications with Her Majesty, etc. and honours',
       'S.38 - Health and Safety', 'S.40 - Personal information',
       'S.41 - Information provided in confidence',
       'S.42 - Legal professional privilege', 'S.43 - Commercial interests',
       'S.44 - Prohibitions on disclosure', 'All EIR exemptions',
       'Unnamed: 25']

## Calculate percentages

Now we have a merged dataframe we can calculate what proportion of the total requests were refused, and what proportion were refused for a particular reason.

In [13]:
#calculate percentages by dividing one column by another
perc_exempted = mergeddf['Total requests where one or more exemptions / exceptions were applied [note 18] [note 19]']/mergeddf['Total requests processed']
#calculate percentages by dividing one column by another
perc_s27 = mergeddf['S.27 - International relations']/mergeddf['Total requests processed']

#add to dataframe
mergeddf['perc_exempted'] = perc_exempted
mergeddf['perc_s27'] = perc_s27

### Sort by those percentages

And we can sort by those columns by using `.sort_values()`

In [14]:
#sort by section 27 percentage - show just the top 3
mergeddf.sort_values('perc_s27', ascending=False).head(3)

,Government body,Total requests received,Total requests processed,"Total requests ""On hold"" or lapsed [note 2]",Total requests still being processed,Total requests handled under EIRs [note 3],Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",...,S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,Unnamed: 25,perc_exempted,perc_s27
10,"Foreign, Commonwealth and Development Office [...",315,229,0,86,2,48,8,1,9,...,9,32,4,1,6,0,0,NaN,0.209607,0.122271
18,UK Export Finance,24,24,0,0,9,9,1,0,0,...,0,0,0,0,3,0,14,NaN,0.375000,0.041667
17,Scotland Office,26,26,0,0,0,3,0,0,0,...,0,3,0,0,1,0,0,NaN,0.115385,0.038462


### The results are in - export them

Not surprisingly, the FCO comes top, with 12% of all requests refused under that exemption.

Let's export the data.

In [15]:
#sort by section 27 percentage and store in new variable 'sorteddata'
sorteddata = mergeddf.sort_values('perc_s27', ascending=False)
#export that variable as a CSV
sorteddata.to_csv("foianalysis.csv")

## Merging vertically (two different periods)

There is another way to merge two datasets: one on top of each other. 

This is something you might need to do if you have datasets from two periods, with the same columns. 

In this case you're not merging, but rather **appending** - and the method to do this is `.append()`.

Let's fetch some data from the period *after* the data we imported before: [Freedom of Information statistics: July to September 2021](https://www.gov.uk/government/statistics/freedom-of-information-statistics-july-to-september-2021)

Because it's the same source, providing data in the same way, we can use the same code - we just need to change the URL to point to the more recent dataset. 

We also need to change the variable name so that we can combine it with the other variable (rather than overwriting it). I'm going to add 'q32021' to the end of the previous variable names to distinguish them. 

In [16]:
#store the URL of the file
odsurlq32021 = "https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1040539/foi-statistics-q3-2021-statistical-tables.ods"
#read the file at that URL, fetching the 12th sheet, starting at the 10th row and capturing 20 rows only
foidataq32021 = pd.read_excel(odsurlq32021, sheet_name=11, header=9, nrows=20)
#check the first few rows
foidataq32021.head(3)

,Government body,Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",S.24 - National security,S.26 - Defence,S.27 - International relations,S.28 - Relations within the United Kingdom,S.29 - The economy,...,"S.35 - Formulation of Government policy, etc.",S.36 - Prejudice to effective conduct of public affairs,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions
0,Attorney General's Office,10,0,0,0,0,0,0,0,0,...,4,1,0,0,2,1,1,0,0,0
1,Cabinet Office [note 4],88,9,0,4,10,2,9,2,0,...,26,5,3,0,23,3,5,14,0,1
2,"Department for Business, Energy and Industrial...",105,13,0,0,3,0,1,0,1,...,11,1,0,1,16,2,2,24,1,45


We can see that the data covers the same categories as we imported from the previous file - but if we weren't sure we could first read in the first row with the sheet title to check that, or import the index sheet (the first sheet in the workbook), or open it manually.

### Add a new column to distinguish the data

Before we combine the two dataframes we need to make sure that we can distinguish them. After all, they both contain the same columns and the same rows. 

The only difference is that the data refers to different periods – but neither dataframe actually *includes that information*. 

We only know that this relates to quarter 3, 2021 because that information was given on the page where we accessed it, and on the titles of the tables (which we skipped when importing). 

So we need to create a new column with that data.

To do that, name a new column after the dataframe name, just as you would when you want to grab data from a column, by putting the column name as a string in square brackets immediately after the dataframe name.

If a column doesn't exist - as in this case - then `pandas` will create it, and fill it with whatever you assign to it. 

Typically we would assign a list which is the same length as the dataframe, but you can also just assign one piece of information - and `pandas` will assume you want that *repeated* down the column.

That's what the code below does.

In [ ]:
#add a new column with a string repeated
foidataq32021['period'] = 'Q3 2021'
#check it's worked - scroll to the far right to see the new column at the end
foidataq32021.head(3)

,Government body,Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",S.24 - National security,S.26 - Defence,S.27 - International relations,S.28 - Relations within the United Kingdom,S.29 - The economy,...,S.36 - Prejudice to effective conduct of public affairs,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,period
0,Attorney General's Office,10,0,0,0,0,0,0,0,0,...,1,0,0,2,1,1,0,0,0,Q3 2021
1,Cabinet Office [note 4],88,9,0,4,10,2,9,2,0,...,5,3,0,23,3,5,14,0,1,Q3 2021
2,"Department for Business, Energy and Industrial...",105,13,0,0,3,0,1,0,1,...,1,0,1,16,2,2,24,1,45,Q3 2021


### Repeat for the other dataset

We repeat the process for the first dataset.

In [ ]:
#add a new column with a string repeated
foidata['period'] = 'Q2 2021'
#check it's worked - scroll to the far right to see the new column at the end
foidata.head(3)

,Government body,Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",S.24 - National security,S.26 - Defence,S.27 - International relations,S.28 - Relations within the United Kingdom,S.29 - The economy,...,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,Unnamed: 25,period
0,Attorney General's Office,9,0,0,0,0,0,1,0,0,...,0,0,2,1,1,0,0,0,NaN,Q2 2021
1,Cabinet Office [note 4],180,41,1,9,14,1,11,1,0,...,7,1,46,9,2,26,0,0,NaN,Q2 2021
2,"Department for Business, Energy and Industrial...",108,12,0,0,4,0,4,0,1,...,0,1,24,3,0,28,2,37,NaN,Q2 2021


### Append the second dataframe to the first

Now we can add the second dataframe to the first, and know which one refers to which period.

We do that by attaching the `.append()` method to the dataframe (`foidata`), indicating that we want to append something to it.

We specify *what* we want to append in the brackets: the name of the other dataframe.

We need to store the results of this process, so use the equals operator to assign it to a new variable, which we call `foi_both`.

In [ ]:
#append the second dataframe to the first, and assign to a new variable
foi_both = foidata.append(foidataq32021)
#show it
foi_both

,Government body,Total requests where one or more exemptions / exceptions were applied [note 18] [note 19],S.22 - Information intended for future publication,S. 22A - Research intended for future publication,"S.23 - Information supplied by, or relating to, bodies dealing with security matters",S.24 - National security,S.26 - Defence,S.27 - International relations,S.28 - Relations within the United Kingdom,S.29 - The economy,...,"S.37 - Communications with Her Majesty, etc. and honours",S.38 - Health and Safety,S.40 - Personal information,S.41 - Information provided in confidence,S.42 - Legal professional privilege,S.43 - Commercial interests,S.44 - Prohibitions on disclosure,All EIR exemptions,Unnamed: 25,period
0,Attorney General's Office,9,0,0,0,0,0,1,0,0,...,0,0,2,1,1,0,0,0,NaN,Q2 2021
1,Cabinet Office [note 4],180,41,1,9,14,1,11,1,0,...,7,1,46,9,2,26,0,0,NaN,Q2 2021
2,"Department for Business, Energy and Industrial...",108,12,0,0,4,0,4,0,1,...,0,1,24,3,0,28,2,37,NaN,Q2 2021
3,"Department for Digital, Culture, Media and Sport",39,5,0,1,0,0,0,0,0,...,1,0,17,1,0,5,0,0,NaN,Q2 2021
4,Department for Education [note 4],124,13,0,0,0,0,0,0,0,...,0,0,81,7,3,5,3,4,NaN,Q2 2021
5,"Department for Environment, Food and Rural Aff...",98,2,0,1,0,0,4,0,0,...,0,0,23,3,0,5,0,60,NaN,Q2 2021
6,Department for International Trade,46,6,0,0,0,8,0,0,0,...,1,0,25,7,0,11,0,0,NaN,Q2 2021
7,Department for Transport [note 4],163,17,0,2,2,0,1,0,0,...,0,2,79,9,1,23,5,7,NaN,Q2 2021
8,Department for Work and Pensions,99,19,0,0,3,0,0,0,0,...,1,0,53,5,0,8,4,1,NaN,Q2 2021
9,Department of Health and Social Care,128,25,0,0,0,0,0,0,0,...,0,0,34,2,1,33,0,1,NaN,Q2 2021


### Export the results

We can now export that so we have a copy.

In [ ]:
foi_both.to_csv("foiexemptions_q2_3_2021.csv")